<a href="https://colab.research.google.com/github/ThiwakornC/Spotter-NBA/blob/main/Spotter_NBAStats_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT Neccessary Libs

In [1]:
import sys
import pandas as pd
import numpy as np
import IPython
from IPython.display import display, Markdown, Latex, HTML, IFrame, JSON, Code, Image, YouTubeVideo, clear_output
import matplotlib.pyplot as plt
import seaborn as sns

Load and Clean Dataset

In [ ]:
df_teamtotal = pd.read_csv("https://raw.githubusercontent.com/ThiwakornC/Spotter-NBA/refs/heads/main/NBA%20Stats_Dataset/Team%20Totals.csv")
df_teamsummary = pd.read_csv("https://raw.githubusercontent.com/ThiwakornC/Spotter-NBA/refs/heads/main/NBA%20Stats_Dataset/Team%20Summaries.csv")
df_playertotal = pd.read_csv("https://raw.githubusercontent.com/ThiwakornC/Spotter-NBA/refs/heads/main/NBA%20Stats_Dataset/Player%20Totals.csv")
df_playershoot = pd.read_csv("https://raw.githubusercontent.com/ThiwakornC/Spotter-NBA/refs/heads/main/NBA%20Stats_Dataset/Player%20Shooting.csv")
#df_teamtotal.head()
df_teamtotal.info()

In [ ]:
df_teamtotal.count()

In [ ]:
df_teamtotal.isna()
#df_teamtotal.query("@df_teamtotal.isna().any(axis=1)", engine='python').reset_index(drop=True)

In [ ]:
#df_teamtotal2 = df_teamtotal.fillna("Unknown")
#df_teamtotal2
threshold = len(df_teamtotal) * 0.5  # Calculate the threshold for 50% NA values
df_teamtotal2 = df_teamtotal.dropna(thresh=threshold, axis=1)  # Drop columns
df_teamtotal2.info()  # Verify the changes (Optional)
df_teamtotal2.count()

In [ ]:
df_teamtotal2.duplicated()
display(df_teamtotal2.duplicated().value_counts())

In [ ]:
# Group data by experience and calculate the average 3P% for each group
grouped_data = df_teamtotal2.groupby('season')['x3p'].mean()
grouped_data2 = df_teamtotal2.groupby('season')['x3pa'].mean()

# Create a figure and a set of subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=True)

# Customize the plot
axes[0].plot(grouped_data.index, grouped_data.values)
axes[0].set_xlabel('Experience (Years)')
axes[0].set_ylabel('3 Poins Made')
axes[0].set_title('Three Poins Made per Experience')
axes[0].grid(True)

axes[1].plot(grouped_data2.index, grouped_data2.values, color='orange')  # Different color
axes[1].set_xlabel('Experience (Years)')
axes[1].set_ylabel('3 Poins Attemp')
axes[1].set_title('Three Poins Attemp per Experience')
axes[1].grid(True)

top_12_x3p = df_teamtotal2.groupby('season')['x3p'].sum().sort_values(ascending=False).head(12)

# Create the bar chart
plt.figure(figsize=(12, 6))  # Adjust figure size if needed
plt.bar(top_12_x3p.index, top_12_x3p.values)

# Customize the plot
plt.xlabel('Season')
plt.ylabel('Total x3p')
plt.title('Top 12 Seasons by Total x3p')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Filter data for the year 2024
data_2024 = df_teamtotal2[df_teamtotal2['season'] == 2024]

# Assuming 'team' is the column containing team names
# Remove rows where 'team' is equal to 'TOT' (or whatever represents the total value)
data_2024 = data_2024[data_2024['tm'] != 'TOT']

# Group by team and sum x3p
team_x3p_2024 = data_2024.groupby('tm')['x3p'].sum()

# Create the bar chart
plt.figure(figsize=(12, 6))  # Adjust figure size as needed
plt.bar(team_x3p_2024.index, team_x3p_2024.values)

# Customize the plot
plt.xlabel('Team')
plt.ylabel('Total x3p')
plt.title('Total x3p by Team in 2024')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()